In [1]:
import argparse
import glob
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import sys
import pickle
import numpy as np
import pandas as pd
from d3mds import D3MDS

In [2]:
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import keras
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, LSTM, InputLayer
from keras import backend as K

Using TensorFlow backend.


In [5]:
class LSTMTimeSeriesPredictor(object):
    def __init__(self, multi_index_cols, time_col, value_col, num_epochs=20):
        self.multi_index_cols = multi_index_cols # tuple
        self.time_col = time_col
        self.value_col = value_col
        self.num_epochs = num_epochs
        self.model = None
            
    def fit(self, X_train, y_train):
        data_train = self._pivot_train_data(X_train, y_train)
        last_day_info = self._compute_last_day_info(data_train)
        
        # Interpolate values
        data_train_interp = data_train.interpolate('index', axis=1, limit_direction='both', inplace=False)
        # Take the differences
        data_train_diff = self._diff_df(data_train_interp)
        # Build supervised dataset
        data_train_diff_shift = self._shift_df(data_train_diff)
        data_train_diff_full = pd.concat([data_train_diff_shift, data_train_diff], axis=1)

        # Scale data to [-1,1]
        scalers = self._build_scalers(data_train_diff_full)
        data_train_scaled = self._apply_scalers(data_train_diff_full, scalers)
        
        # Build lstm model
        self.model = self._build_lstm_model()
        
        # Train lstm model
        model_file = 'lstm_model_%d_epochs.h5' % self.num_epochs
        if os.path.exists(model_file):
            self._load_prev_model(model_file)
        else:
            for e in range(self.num_epochs):
                print('On Epoch %d' % e)
                
                for i, (idx, df) in enumerate(data_train_scaled.groupby(level=[0,1])):
                    print('Processing series: ', idx)
                    print(last_day_info[idx])
                    last_ind = last_day_info[idx]['last_ind']
                    self._train_lstm(self.model, data_train_scaled.loc[idx][:, None],
                                     batch_size=1, nb_epoch=1, last_ind=last_ind)
                    
                self.model.save('lstm_model_%d_epochs.h5' % self.num_epochs)

        # Save training data + (last_day_info, scalers) to .pkl file
        train_data_dict = {'data_train': data_train, 'data_train_scaled': data_train_scaled,
                           'scalers': scalers, 'last_day_info': last_day_info}        
        with open('train_data.pkl', 'wb') as f:
            pickle.dump(train_data_dict, f)
        
    def predict(self, X_test):
        data_test = X_test.copy()
        data_test.set_index(self.multi_index_cols, drop=True, inplace=True)

        preds_test = []
        for i, (idx, df) in enumerate(data_test.groupby(level=[0,1], sort=False)):
            for day in df['day']:
                print(idx, day)
                pred = self.predict_single_ts(idx, day)
                print(pred)
                preds_test.append(pred)                
      
        data_test.reset_index(inplace=True)
        
        return preds_test

    def predict_single_ts(self, idx, target_day):
        # Load training data + (last_day_info, scalers) from .pkl file
        data_train, data_train_scaled, scalers, last_day_info = self._load_train_dict('./train_data.pkl')
        
        # forecast the entire training dataset to build up state for forecasting
        last_ind, last_day = last_day_info[idx]['last_ind'], last_day_info[idx]['last_day']

        df = data_train.loc[idx][0:last_ind+1]
        df_scaled = data_train_scaled.loc[idx][0:last_ind+1]
        #print(df_scaled)
        scaler = scalers[idx]

        self.model.reset_states()
        res = self.model.predict(df_scaled[:, None, None], batch_size=1)   

        if target_day <= last_day:        
            print('target_day less than last_day')
            target_ind = np.where(data_train.columns == target_day)[0]
            return res[target_ind]

        target_ind = last_ind+(target_day-last_day)
        #print(target_ind)
        for i in range(last_ind+1, target_ind+1):
            #print(res[-1])
            val = self.model.predict(res[-1][:, None, None], batch_size=1)
            #print(val)
            res = np.append(res, val[0])[:, None]

        res_inv_scale = scaler.inverse_transform(res)
        pred_train = df[:, None] + res_inv_scale[0:last_ind+1]
        remainder = (df.values[-1] + np.cumsum(res_inv_scale[last_ind+1:]))[:,None]
        return remainder[-1]
        
    def _load_train_dict(self, pkl_file):
        with open(pkl_file, 'rb') as f:
            d = pickle.load(f)
        return d['data_train'], d['data_train_scaled'], d['scalers'], d['last_day_info']
    
    def _load_prev_model(self, model_file):
        self.model.load_weights(model_file)
   
    def _pivot_train_data(self, X_train, y_train):
        data_train = X_train.copy()
        data_train[self.value_col] = y_train
        data_train = data_train.pivot_table(index=self.multi_index_cols,
                                            columns=self.time_col,
                                            values=self.value_col)
        return data_train
    
    def _compute_last_day_info(self, X_train):
        last_day_info = {}

        for i, (idx, df) in enumerate(X_train.groupby(level=[0,1])):
            #print(i, (idx, df))
            last_ind = np.where(X_train.notnull().iloc[i,:]==True)[0][-1]
            last_day =  X_train.columns[last_ind]

            last_day_info[idx] = {}
            last_day_info[idx]['last_ind'] = last_ind
            last_day_info[idx]['last_day'] = last_day
            
        return last_day_info
    
    # Create shifted form of each timeseries - (creates supervised version of timeseries problem)
    def _shift_df(self, df, lag=1):    
        shifted_df = df.shift(lag, axis=1)
        shifted_df.fillna(0, inplace=True)
        return shifted_df
    
    def _diff_df(self, df):
        df_diff = df.diff(periods=-1, axis=1)
        df_diff.fillna(0, inplace=True)
        return df_diff
    
    def _build_scalers(self, train_df):
        scalers = {}
        for i, (idx, df) in enumerate(train_df.groupby(level=[0,1])):
            #print(i, (idx, df))
            scaler = MinMaxScaler(feature_range=(-1, 1))
            scaler = scaler.fit(df.T)
            scalers[idx] = scaler

        return scalers
    
    def _apply_scalers(self, df_in, scalers):
        df_out = df_in.copy()
        for i, (idx, df) in enumerate(df_in.groupby(level=[0,1])):
            #print('before scale', df_in.loc[idx])
            df_out.loc[idx] = scalers[idx].transform(df_in.loc[idx].reshape(1,-1))
            #print('after scale', df_out.loc[idx])

        return df_out
    
    def _build_lstm_model(self, batch_size=1, num_time_steps=1, num_feat=1, neurons=5):
        model = Sequential()
        model.add(LSTM(neurons, batch_input_shape=(batch_size, num_time_steps, num_feat), stateful=True))
        model.add(Dense(1))
        opt = keras.optimizers.Adam(lr=1e-4)
        model.compile(loss='mae', optimizer=opt)
        return model

    def _train_lstm(self, model, train, batch_size, nb_epoch, last_ind):
        #TODO: hard-coded 304 (needs to be addressed)
        X, y = train[0:last_ind+1, 0:1], train[304:(304+last_ind+1), 0:1]
        print(X.shape, y.shape)
        X = X.reshape(X.shape[0], 1, X.shape[1])
        #print(X.shape)
        for i in range(nb_epoch):
            model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
            model.reset_states()
        return model

In [6]:
# Fix random seed for reproducibility
np.random.seed(42)

In [7]:
dataset_root = '../../'
model_dir = './models'

In [8]:
print('Load DATA')    
data_path = glob.glob(os.path.join(dataset_root, "*_dataset"))[0]
problem_path = glob.glob(os.path.join(dataset_root, "*_problem"))[0]
d3mds = D3MDS(data_path, problem_path)

Load DATA


/data/sptemp10/d3m/staging/LL1_736_population_spawn/LL1_736_population_spawn_solution/src/d3mds.py:31: UserWarning: the datasetSchemaVersions in the API and datasetDoc do not match !!!!!!!
  warnings.warn("the datasetSchemaVersions in the API and datasetDoc do not match !!!!!!!")
/data/sptemp10/d3m/staging/LL1_736_population_spawn/LL1_736_population_spawn_solution/src/d3mds.py:199: UserWarning: the problemSchemaVersions in the API and datasetDoc do not match !!!!!!!
  warnings.warn("the problemSchemaVersions in the API and datasetDoc do not match !!!!!!!")


In [9]:
print('Load train data')
df_train = d3mds.get_train_data()
targets_train = d3mds.get_train_targets()

Load train data


In [10]:
lstm_ts_predictor = LSTMTimeSeriesPredictor(multi_index_cols=['species', 'sector'], 
                                            time_col='day',
                                            value_col='count')

In [11]:
lstm_ts_predictor.fit(df_train, targets_train)

/home/pkhorrami/.envs/d3m_population_spawn/lib/python3.6/site-packages/ipykernel_launcher.py:155: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [12]:
print('Load test data')
df_test = d3mds.get_test_data()
targets_test = d3mds.get_test_targets()

Load test data


In [13]:
preds = lstm_ts_predictor.predict(df_test)

('cas9_VBBA', 'S_3102') 330
[ 44553.11328125]
('cas9_VBBA', 'S_3102') 360
[ 54817.2734375]
('cas9_VBBA', 'S_4102') 328
[ 46734.90625]
('cas9_VBBA', 'S_4102') 360
[ 45368.828125]
('cas9_VBBA', 'S_5102') 329
[ 45066.13671875]
('cas9_VBBA', 'S_5102') 362
[ 60473.81640625]
('cas9_VBBA', 'S_6102') 333
[ 71017.8125]
('cas9_VBBA', 'S_6102') 363
[ 102680.921875]
('cas9_VBBA', 'S_7102') 304
[ 90699.234375]
('cas9_VBBA', 'S_7102') 306
[ 90012.96875]
('cas9_FAB', 'S_5002') 332
[ 10632.08203125]
('cas9_FAB', 'S_5002') 362
[ 13476.20507812]
('cas9_FAB', 'S_6002') 332
[ 9844.49902344]
('cas9_FAB', 'S_6002') 361
[ 10563.65820312]
('cas9_FAB', 'S_7002') 332
[ 10302.68554688]
('cas9_FAB', 'S_7002') 358
[ 11970.24804688]
('cas9_FAB', 'S_8002') 329
[ 6594.29248047]
('cas9_FAB', 'S_8002') 361
[ 8722.60253906]
('cas9_FAB', 'S_9002') 329
[ 9889.09765625]
('cas9_FAB', 'S_9002') 362
[ 12048.38769531]
('cas9_FAB', 'S_0102') 330
[ 11059.23730469]
('cas9_FAB', 'S_0102') 362
[ 13002.609375]
('cas9_FAB', 'S_1102')

[ 20259.46289062]
('cas9_MBI', 'S_9691') 328
[ 15374.5859375]
('cas9_MBI', 'S_9691') 360
[ 18308.54296875]
('cas9_MBI', 'S_0791') 328
[ 12900.29394531]
('cas9_MBI', 'S_0791') 362
[ 15110.90234375]
('cas9_MBI', 'S_1791') 330
[ 14368.36914062]
('cas9_MBI', 'S_1791') 362
[ 18997.12890625]
('cas9_MBI', 'S_2791') 333
[ 15240.02539062]
('cas9_MBI', 'S_2791') 357
[ 15842.24902344]
('cas9_MBI', 'S_3791') 332
[ 16283.62402344]
('cas9_MBI', 'S_3791') 358
[ 20062.03320312]
('cas9_MBI', 'S_4791') 330
[ 8759.49121094]
('cas9_MBI', 'S_4791') 360
[ 8547.85644531]
('cas9_MBI', 'S_5791') 328
[ 10883.90136719]
('cas9_MBI', 'S_5791') 360
[ 11993.59082031]
('cas9_MBI', 'S_6791') 331
[ 13287.9375]
('cas9_MBI', 'S_6791') 363
[ 14506.46484375]
('cas9_MBI', 'S_7791') 332
[ 14348.76953125]
('cas9_MBI', 'S_7791') 362
[ 16684.47460938]
('cas9_MBI', 'S_8791') 332
[ 14712.97851562]
('cas9_MBI', 'S_8791') 361
[ 16172.57910156]
('cas9_MBI', 'S_9791') 332
[ 14635.55761719]
('cas9_MBI', 'S_9791') 358
[ 17119.82617188]

In [14]:
preds

[array([ 44553.11328125], dtype=float32),
 array([ 54817.2734375], dtype=float32),
 array([ 46734.90625], dtype=float32),
 array([ 45368.828125], dtype=float32),
 array([ 45066.13671875], dtype=float32),
 array([ 60473.81640625], dtype=float32),
 array([ 71017.8125], dtype=float32),
 array([ 102680.921875], dtype=float32),
 array([ 90699.234375], dtype=float32),
 array([ 90012.96875], dtype=float32),
 array([ 10632.08203125], dtype=float32),
 array([ 13476.20507812], dtype=float32),
 array([ 9844.49902344], dtype=float32),
 array([ 10563.65820312], dtype=float32),
 array([ 10302.68554688], dtype=float32),
 array([ 11970.24804688], dtype=float32),
 array([ 6594.29248047], dtype=float32),
 array([ 8722.60253906], dtype=float32),
 array([ 9889.09765625], dtype=float32),
 array([ 12048.38769531], dtype=float32),
 array([ 11059.23730469], dtype=float32),
 array([ 13002.609375], dtype=float32),
 array([ 10840.42871094], dtype=float32),
 array([ 13043.82617188], dtype=float32),
 array([ 13813

In [15]:
targets_test

array([[ 42341],
       [ 46166],
       [ 60957],
       [ 60236],
       [ 56001],
       [ 54571],
       [ 57367],
       [ 60078],
       [ 90250],
       [ 91020],
       [  9074],
       [  9025],
       [  9612],
       [  9498],
       [  9040],
       [  8737],
       [  6321],
       [  6375],
       [  9047],
       [  9386],
       [  9753],
       [  8827],
       [ 10878],
       [ 11105],
       [ 13155],
       [ 12554],
       [ 10112],
       [ 10693],
       [ 12128],
       [ 12314],
       [ 13020],
       [ 13300],
       [ 13288],
       [ 13478],
       [ 14990],
       [ 15100],
       [ 34530],
       [ 36260],
       [ 46849],
       [ 51966],
       [ 48680],
       [ 42770],
       [ 27480],
       [ 27208],
       [ 34902],
       [ 39265],
       [ 43453],
       [ 46895],
       [ 39156],
       [ 39519],
       [ 32073],
       [ 36765],
       [ 30650],
       [ 29969],
       [ 29734],
       [ 31693],
       [ 29183],
       [ 30004],
       [ 28136

In [16]:
mean_absolute_error(targets_test, preds)

8178.3299519750808